In [1]:
import numpy as np
from functools import lru_cache
from collections import defaultdict
import pathlib
import pickle


def get_sim_func(imgs):
    @lru_cache(maxsize=None)
    def sim(a, b, d):
        height = min(imgs[a].shape[0], imgs[b].shape[0])
        width = min(imgs[a].shape[1], imgs[b].shape[1])
        if d == 'r':
            return imgs[a][-1, :, :][:height]
            rhs = imgs[b][0, :, :][:height]
        elif d == 'c':
            lhs = imgs[a][:, -1, :][:, :width]
            rhs = imgs[b][:, 0, :][:, :width]
        else:
            raise
        return (lhs == rhs).sum()
    return sim
        

def sim(a, b, d):
    if d == 'r':
        lhs = a[-1, :, :][:b.shape[0]]
        rhs = b[0, :, :][:a.shape[0]]
    elif d == 'c':
        lhs = a[:, -1, :][:, :b.shape[1]]
        rhs = b[:, 0, :][:, :a.shape[1]]
    else:
        raise
    return (lhs == rhs).sum()


def total_cost(imgs, rows, cols):
    cost = 0
    n = rows*cols
    for i in range(len(imgs)):
        if (1 + 1)%cols > 0:
            cost += sim(imgs[i], imgs[i + 1], 'r')
        if i + cols < n:
            cost += sim(imgs[i], imgs[i + cols], 'c')
    return cost


def get_coeff(imgs, rows, cols):
    """
    \sum_{d}S_{abd}D_{ijd} を返す
    """
    n = rows*cols
    coeff = defaultdict(float)
    for a in range(len(imgs)):
        for b in range(len(imgs)):
            for i in range(rows*cols):
                if (i + 1)%cols > 0:
                    coeff[a, b, i, i + 1] = sim(imgs[a], imgs[b], 'r')
                if i + cols < n:
                    coeff[a, b, i, i + cols] = sim(imgs[a], imgs[b], 'c')
    return coeff

In [2]:
IMG_DIR = '../data/shred'

imgs = sorted(pathlib.Path(IMG_DIR).glob('*.pkl'))

In [3]:
with open(imgs[12], 'rb') as i_:
    i = pickle.load(i_)

5